In [1]:
import os, sys
from dask import compute, persist
from dask.distributed import Client, progress
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split #,KFold,RandomizedSearchCV, GridSearchCV, HyperbandSearchCV #, RandomizedSearchCV
import matplotlib.pyplot as plt
import dask_xgboost as dxgb
import xgboost as xgb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import KFold,RandomizedSearchCV , GridSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
import joblib
import numpy as np

In [2]:
# import dask_jobqueue
# from dask_jobqueue import SLURMCluster
# cluster = SLURMCluster(cores=32,
#                        processes=16,
#                        memory="128GB",
#                        walltime="24:00:00",
#                        queue="workq",
#                        interface="ipogif0",

#                        project="k1423"
#                       )
# print(cluster.job_script())
# cluster.scale(jobs=8)



In [5]:
# client = Client(cluster)
client = Client()

In [6]:

all_files = [x for x in os.listdir("data/EDITED_CELL_LINES_FILES_COMPLETE/") if ".csv" in x and x != "subchallenge_1_template_data.csv"  ]
# files = ['184B5.csv',
# 'BT483.csv',
# 'HCC1428.csv',
# 'HCC1806.csv',
# 'HCC202.csv',
# 'Hs578T.csv',
# 'MCF12A.csv',
# 'MDAMB231.csv',
# 'MDAMB468.csv',
# 'SKBR3.csv',
# 'UACC3199.csv',
# 'ZR751.csv']
#target cell lines subch one
target_cell_lines = ['AU565', 'EFM19', 'HCC2218', 'LY2', 'MACLS2', 'MDAMB436']

#targert genes subchallenge one
target_genes = ['p.ERK', 'p.Akt.Ser473.', 'p.S6', 'p.PLCg2_c', 'p.HER2_c']


184B5.csv
BT483.csv
HCC1428.csv
HCC1806.csv
HCC202.csv
Hs578T.csv
MCF12A.csv
MDAMB231.csv
MDAMB468.csv
SKBR3.csv
UACC3199.csv
ZR751.csv


In [ ]:
df = []

for m in all_files:
#     print (x)
    df_temp = dd.read_csv("data/EDITED_CELL_LINES_FILES_COMPLETE/%s"%(m))
    df_temp.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
#     df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df.append(df_temp)
    
for x in os.listdir("data"):
#     print (x)
    df_temp = dd.read_csv("data/%s"%(x))
#     df_temp.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
#     df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df.append(df_temp)

In [7]:
ddf = dd.concat(df,axis=0)

In [8]:
print ("filling NA")
# y = y.compute()
with ProgressBar():
    for m in ["p.HER2","p.PLCg2"] :
#         print (df[m])
        
        ddf["%s_c"%(m)] =ddf[m].fillna(ddf[m].mean() )#, inplace=True )

filling NA


In [9]:
ddf = dd.get_dummies(ddf.categorize()).persist()

In [10]:
# ddf = ddf.drop(["p.HER2","p.PLCg2","cellID","fileID"],axis=1)
ddf = ddf.drop(["p.HER2","p.PLCg2"],axis=1)

In [11]:
# rounds = {}
# genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.',
#         'p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3',
#         'p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1',
#         'p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

# for x in range(len (genes)):
# #     print (x)
#     my_list = ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.',
#                'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK',
#                'p.MAP2K3','p.MAPKAPK2','p.MEK',   'p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK',
#                'p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

#     if x+1 <len (genes):
# #         my_list = genes 
# #         print (x,genes[x], genes[x+1])
#         my_list.pop(x)
# #         my_list.pop(x)
# #         my_list.pop(x+1)
# #         print ( my_list )
# #         rounds[x] = { "test":[genes[x], genes[x+1]] , "train":my_list}
#         rounds[x] = { "test":[genes[x] ], "train":my_list}
# #         y = df[[genes[x], genes[x+1]]]

In [12]:
# feats=  ['time','treatment_EGF', 'treatment_full', 'treatment_iEGFR', 'treatment_iPI3K','treatment_iPKC', 'treatment_iMEK']

# for k in  rounds :
#     for ff in feats: 
#         rounds[k]["train"].append(ff)
        

In [ ]:
## for sub challenge one 
train_ddf = ddf[~ddf["cell_line"].isin(target_cell_lines)]
test_ddf = ddf[ddf["cell_line"].isin(target_cell_lines)]
X_train = train_ddf.drop(target_genes,axis=1)
X_train = X_train.to_dask_array(lengths=True)
y_train = train_ddf[target_genes]
X_test = test_ddf.drop(target_genes,axis=1)
X_test = X_test.to_dask_array(lengths=True)

In [ ]:
my_target_ditionary = {'ERK':'p.ERK', 'AKT':'p.Akt.Ser473.', 'S6':'p.S6', 'HER':'p.HER2_c', 'PLCG2':'p.PLCg2_c'}

In [ ]:
for key in my_target_ditionary.keys():
    
    y2 = y_train[my_target_ditionary[key]]
    
    y2 = y2.to_dask_array(lengths=True)
    print ("split")
#     X_train, X_test, y_train, y_test = train_test_split(X,y2,test_size=0.3, random_state=101)
    
    print ("scaling")
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    base_model = dxgb.XGBRegressor(objective='reg:squarederror',tree_method='hist',
                               verbosity=3,n_jobs=-1,n_estimators=1000,
                               learning_rate=0.1  ,max_depth=0,max_leaves=64,grow_policy='lossguide') 
    with joblib.parallel_backend('dask'):
        base_model.fit(X_train, y_train.flatten())
        base_model.save_model('%s_xgboost.model'%(key))

    predictions = base_model.predict( X_test)
    predictions = client.persist(predictions)

    p = predictions.to_dask_dataframe(columns=y.columns)
    p.to_csv("my_result_%s_subOne"%(my_target_ditionary[key]))


In [13]:
# X = ddf[rounds[0]["train"]]
# y = ddf[rounds[0]["test"]]


split


In [ ]:
# print ("scaling")
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)

In [2]:
base_model = dxgb.XGBRegressor(objective='reg:squarederror',tree_method='hist',
                               verbosity=3,n_jobs=-1,n_estimators=1000,
                               learning_rate=0.1  ,max_depth=0,max_leaves=64,grow_policy='lossguide' )

In [ ]:
# with joblib.parallel_backend('dask'):
#     base_model.fit(X_train, y_train.flatten())
# # base_model.save_model('base_line_no_max_deph_lr_%f_%i.model'%(lr,leaves))

# predictions = base_model.predict( X_test)
# predictions = client.persist(predictions)
# print ("########")
# # print ("lr :%f , leaves:%i " %(lr,leaves))
# print ("R^2:",r2_score(y_test.compute(), predictions.compute()))
# print ("MAE:",mean_absolute_error(y_test.compute(), predictions.compute()))
# print ("MSE:",mean_squared_error(y_test.compute(), predictions.compute()))



In [3]:
base_model = xgb.Booster()

In [ ]:
dxgb.core.xgb.Booster(model_file=)